## Imports

In [ ]:
import numpy as np
import tiktoken  # Tokenizer-Bibliothek für GPT-2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

# 1. Tokenisierung:
- Tokenisierung ist der Prozess der Umwandlung von Text in kleinere Einheiten (Tokens).
- Tokens können Zeichen ("a", "b"), Wörter ("Hallo", "Baum") oder Subwörter ("Ha", "Ba") sein.
- LLMs arbeiten mit Token-Sequenzen anstelle von Rohtext → Tokenisierung ist daher das Vorbereiten des Textes für die Eingabe in das LLM :)




#### 1.1 Datensatz erstellen:

In [ ]:
class DatasetForGPT(Dataset):
    def __init__(self, max_length, stride, tokenizer, txt):
        """
        Erstellt ein Dataset aus einem Text für ein GPT-basiertes Modell.

        Parameter:
        - txt: Der Eingabetext als String. (z.B. ein Opensource Buch der Seite Projekt Gutenberg https://www.projekt-gutenberg.org/)
        - tokenizer: Der GPT2-Tokenizer zur Tokenisierung des Textes.
        - max_length: Maximale Länge einer Token-Sequenz.
        - stride: Schrittweite für die Erstellung überlappender Sequenzen.
        """
        self.input_ids = []  # Liste zur Speicherung der Eingabe-Tokens
        self.target_ids = []  # Liste zur Speicherung der Ziel-Tokens (verschobene Sequenz)

        # Den gesamten Text in Tokens umwandeln
        token_ids = tokenizer.encode(txt, allowed_special={"<|endoftext|>"})

        # Erzeuge überlappende Sequenzen aus den Token-IDs (siehe Erklärung 1)
        for i in range(0, len(token_ids) - max_length, stride):
            input_chunk = token_ids[i:i + max_length]  # Eingabesequenz
            target_chunk = token_ids[i + 1: i + max_length + 1]  # Zielsequenz (verschoben um 1 Token -> siehe Bild 1: Erklärung 1 )

            # Speichern der Tensor-Repräsentation der Sequenzen
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))

    def __len__(self):
        """ Gibt die Anzahl der Trainingsbeispiele zurück."""
        return len(self.input_ids)

    def __getitem__(self, idx):
        """ Gibt ein Trainingsbeispiel bestehend aus (Eingabe, Ziel) zurück."""
        return self.input_ids[idx], self.target_ids[idx]


##### Erklärung 1
Das bedeutet:
- Das Modell bekommt eine Eingabesequenz (input_chunk), z. B. "LLMs learn to predict one"
- Die Zielsequenz (target_chunk) ist dann dieselbe Sequenz, aber um ein Token nach rechts verschoben, sodass das Modell lernen soll, das nächste Token vorherzusagen.

→ Genau wie im Bild (Quelle: Raschka 2025)
###### Bild 1 Sliding Window Approach
![Image 2 Sliding Window Approach](images/Image1_sliding_window_approach.png)

Das Modell sieht nur den bisherigen Kontext (blau markiert).
Das Modell soll das nächste Wort (rot markiert) vorhersagen.
Es kann zukünftige Wörter nicht direkt sehen, sondern muss sie aus den bisherigen Token ableiten. Während des Trainings bekommt das Modell eine Sequenz (input_chunk) als Eingabe und versucht, das nächste Token (target_chunk) vorherzusagen. Durch viele Wiederholungen lernt das Modell dann grammatische Strukturen, Satzbedeutungen und sogar komplexe Zusammenhänge.

#### 1.2 Dataloader erzeugen:

Der DataLoader hilft, die Daten effizient für das LLM bereitzustelle.


In [ ]:
def create_dataloader(txt, batch_size=4, max_length=256,
                      stride=128, shuffle=True, drop_last=True, num_workers=0):
    """
    Erstellt einen DataLoader für das Training eines LLMs.

    Parameter:
    - txt: Eingabetext als String.
    - batch_size: Anzahl der Samples pro Batch.
    - max_length: Maximale Token-Sequenzlänge.
    - stride: Schrittweite für die Erzeugung überlappender Sequenzen.
    - shuffle: Ob die Reihenfolge der Sequenzen zufällig gemischt wird.
    - drop_last: Ob das letzte Batch verworfen wird, falls es unvollständig ist.
    - num_workers: Anzahl der Threads für die Datenverarbeitung.

    Rückgabe:
    - Ein DataLoader-Objekt für das Training.
    """
    # Initialisiere den GPT-2 Tokenizer
    tokenizer = tiktoken.get_encoding("gpt2")

    # Erstelle das Dataset
    dataset = DatasetForGPT(max_length, stride, tokenizer, txt)

    # Erstelle den DataLoader aud der torch Lib (Erklärung 2):
    dataloader = DataLoader(
        dataset, batch_size=batch_size, shuffle=shuffle, drop_last=drop_last, num_workers=num_workers)
    #Shuffle sorgt für zufällige Reihenfolge -> Damit das Modell nicht immer dieselbe Reihenfolge der Daten sieht (wichtig für das Training).
    # Drop_last vermeidet ungleich große Batches
    # Falls die Anzahl der Daten nicht genau durch batch_size teilbar ist, werden unvollständige Batches verworfen.

    return dataloader


##### Erklärung 2:

Ein DataLoader bereitet die Daten für das Modelltraining vor. In diesem Fall fasst der Data Loader mehrere Sequenzen (des Sliding Window Approaches) zu Batches zusammen:

Das Modell verarbeitet nicht einen einzelnen Satz nach dem anderen, sondern mehrere Sequenzen gleichzeitig (z. B. 4 auf einmal, wenn batch_size=4 ist).
###### Beispiel:
**Batch 1:**
- Eingabe:  `["LLMs learn to predict", "learn to predict one", "to predict one word", "predict one word at"]`
- Ziel:     `["learn to predict one", "to predict one word", "predict one word at", "one word at a"]`

**Batch 2:**
- Eingabe:  `["one word at a", "word at a time", "at a time <|endoftext|>", "..."]`
- Ziel:     `["word at a time", "at a time <|endoftext|>", "...", "..."]`




#### 1.3 Code testen

In [ ]:
with open("test_text", "r") as f:
    test_text = f.read()

# DataLoader erstellen
dataloader = create_dataloader(test_text, batch_size=2, max_length=6, stride=3)

# Ersten Batch ausgeben
for batch in dataloader:
    inputs, targets = batch
    print("\n=== Erster Batch ===")
    print("Input Shape:", inputs.shape)   # Erwartet: (batch_size, max_length)
    print("Target Shape:", targets.shape) # Erwartet: (batch_size, max_length)
    print("\nEingabe Batch:", inputs)
    print("Ziel Batch:", targets)
    break  # Nur den ersten Batch ausgeben



#### 1.4 Zusammenfassung Tokenisierung
- Das DatasetForGPT erzeugt nur die überlappenden Sequenzen (Sliding Window Approach, siehe Bild 1) und speichert bereits alle Input- und Target Sequenzen
- Der DataLoader fasst diese dann in Batches zusammen, die dann im Training verwendet werden können.

#### Sources:

Raschka, Sebastian (2025): Build a Large Language Model (from scratch). Shelter Island: Manning (From scratch series). Online verfügbar unter https://ebookcentral.proquest.com/lib/kxp/detail.action?docID=31657639.